# Recognizing Depth in Autonomous Driving
**[링크는 여기](https://towardsdatascience.com/depth-prediction-autonomous-driving-18d05ff25dd6)**


Recognizing Depth in Autonomous Driving

자율주행 깊이 인식
이 글은 추가 카메라나 센서를 사용하지 않고도 새로운 자율주행 모델의 개발에 도움이 되는 차량에 의해 포착된 이미지 시퀀스의 몇 가지 최첨단 방법을 심층적으로 설명할 것이다.

이전 기사에서 언급했듯이 "자율주행은 어떻게 작동하는가? "SLAM에 대한 소개"에는 차량이 주행하는 동안 정보를 캡처하는 데 사용되는 많은 센서가 있습니다. 포착된 다양한 측정에는 속도, 위치, 깊이, 열 등이 포함됩니다. 이러한 측정은 차량이 준수할 수 있도록 모션 모델을 훈련하고 활용하는 피드백 시스템에 공급됩니다. 이 글은 LiDAR 센서에 의해 종종 포착되는 깊이 예측에 초점을 맞춘다. LiDAR 센서는 레이저를 사용하여 반사된 빛을 센서로 측정하여 물체로부터의 거리를 캡처합니다. 그러나 LiDAR 센서는 일상적인 드라이버에는 적합하지 않습니다. 그렇지 않으면 어떻게 깊이를 측정할 수 있을까요? 내가 설명할 최첨단 방법은 깊이를 측정하기 위해 한 프레임에서 다음 프레임까지 픽셀의 차이 또는 차이를 사용하는 비지도 딥 러닝 접근법이다.

대부분의 이미지는 참조 중인 원본 문서에서 가져온 것이며 나만의 이미지 캡션을 만들거나 만든 것이 아닙니다.

## 모노데프 2

[1]의 저자들은 깊이와 포즈 네트워크의 조합을 사용하여 단일 프레임의 깊이를 예측하는 방법을 개발했다. 이들은 두 네트워크를 훈련시키기 위해 일련의 프레임과 몇 가지 손실 함수에 대한 아키텍처를 훈련함으로써 이를 수행한다. 이 방법은 훈련을 위해 사실 데이터 세트를 필요로 하지 않는다. 대신, 그들은 훈련 신호를 제공하기 위해 이미지 시퀀스에서 연속적인 시간 프레임을 사용한다. 학습을 제약하기 위해 포즈 추정 네트워크를 사용한다. 모델은 입력 이미지와 포즈 네트워크 및 깊이 네트워크의 출력에서 재구성된 이미지 간의 차이에 대해 학습된다. 재구성 프로세스는 나중에 더 자세히 설명됩니다. [1]의 주요 기여는 다음과 같다.
1. 중요하지 않은 픽셀에 초점을 맞추는 자동 마스크 기법
2. 깊이 맵을 이용한 측광학적 재구성 오류 수정
3. 다단계 깊이 추정

### 건축(Architecture)

이 논문의 접근 방식은 깊이 네트워크와 자세 네트워크를 사용한다. 깊이 네트워크는 고전적인 U-Net[2] 인코더-디코더 아키텍처이다. 인코더는 사전 훈련된 ResNet 모델입니다. 깊이 디코더는 시그모이드 출력을 깊이 값으로 변환하는 이전 작업과 유사합니다.

![Sample image of U-Net [2].](paper_1.png)  
> Sample image of U-Net [2].  

저자들은 단일 6DoF 상대적인 포즈 또는 회전과 번역을 예측하기 위해 두 개의 컬러 이미지를 입력으로 가져가도록 수정된 ResNet18의 포즈 네트워크를 사용한다. 포즈 네트워크는 일반적인 스테레오 쌍이 아니라 시간 프레임을 영상의 쌍으로 사용한다. 이것은 순서에서 다른 이미지의 관점에서 대상 이미지의 모양을 예측합니다. 즉, 이전 프레임 또는 이후 프레임입니다.

![6-DoF. Image from Wikipedia.](paper_2.png)  
> 6-DoF. Image from Wikipedia.  

### 트레이닝(Training)

아래 그림은 아키텍처의 교육 프로세스를 보여줍니다.
![Images taken from KITTI and [1].](paper_3.png)
> Images taken from KITTI and [1].  

### 측광학적 재구성 오류

목표 이미지는 프레임 0에 있고 우리의 예측 프로세스에 사용되는 이미지는 프레임 전 또는 이후의 프레임일 수 있으므로 프레임+1 또는 프레임-1이다. 손실은 대상 영상과 재구성된 대상 영상 간의 유사성을 기반으로 합니다. 재구성 프로세스는 포즈 네트워크를 사용하여 소스 프레임(프레임+1 또는 프레임-1)에서 변환 매트릭스를 계산하는 것으로 시작한다. 이것은 우리가 회전과 변환에 대한 정보를 사용하여 소스 프레임에서 대상 프레임으로의 매핑을 계산하고 있다는 것을 의미합니다. 그런 다음 대상 이미지에 대한 깊이 네트워크에서 예측된 깊이 맵과 포즈 네트워크에서 예측된 변환 매트릭스를 사용하여 본질적 매트릭스 K가 있는 카메라로 투영하여 재구성된 대상 이미지를 얻는다. 이 프로세스를 수행하려면 먼저 깊이 맵을 3D 포인트 클라우드로 변환한 다음 카메라 본질을 사용하여 3D 위치를 2D 포인트로 변환해야 합니다. 결과 지점은 대상 영상에서 선형 보간하는 샘플링 그리드로 사용됩니다.

이 손실의 목표는 포즈와 깊이가 모두 필요한 대상 영상과 재구성된 대상 영상 간의 차이를 줄이는 것입니다.
![Photometric Loss function from [1].](paper_4.png)
> Photometric Loss function from [1].    

일반적으로 유사한 방법은 프레임+1 및 프레임-1과 같은 각 소스 이미지에 대한 재투영 오류를 함께 평균화한다. 그러나 픽셀이 이러한 프레임 중 하나에서 보이지 않지만 이미지 경계에 가깝거나 막혔기 때문에 대상 프레임에 있는 경우 광도 오차 범칙이 부당하게 높을 것이다. 이와 관련된 문제를 해결하기 위해 대신 모든 소스 이미지에 대해 최소 측광학적 오류를 수행합니다.
![Benefit of using minimum photometric error. Pixel area circled are occluded. Image from [1].](paper_5.png)
> Benefit of using minimum photometric error. Pixel area circled are occluded. Image from [1].  

### 자동 마스킹

최종 측광학적 손실은 카메라가 정적 장면에서 움직이고 있다는 가정에서의 변화와 관련된 문제를 해결하는 마스크에 의해 곱된다. 예를 들어, 물체가 카메라 또는 다른 물체가 움직이는 동안 카메라가 멈춘 것과 유사한 속도로 움직이고 있다. 이 상황의 문제는 깊이 맵이 무한한 깊이를 예측한다는 것이다. 작성자는 이 문제를 한 프레임에서 다음 프레임으로 모양을 변경하지 않는 픽셀을 필터링하는 자동 마스크 방법으로 해결합니다. 이들은 대상 영상과 재구성된 대상 영상 사이의 최소 광도 오차가 대상 영상과 소스 영상의 최소 광도 오차보다 작은 경우 1인 이진법을 사용하여 마스크를 생성하고, 그렇지 않으면 0이다.  

![Auto-masking generation in [1] where Iverson bracket returns 1 if true and 0 if false.](paper_6.png)

카메라가 정지 상태이면 영상의 모든 픽셀이 마스킹됩니다. 개체가 카메라와 동일한 속도로 이동하면 이미지에 있는 고정 개체의 픽셀이 마스킹됩니다.  

### 다중 척도 추정

저자들은 각각의 규모에서 개별적인 손실을 합친다. 이들은 낮은 해상도 깊이 맵을 더 높은 입력 이미지 해상도로 업샘플링한 다음 더 높은 입력 해상도에서 광도 측정 오류를 다시 투영, 재샘플링 및 계산한다. 저자는 이것이 대상 이미지의 동일한 목표, 정확한 고해상도 재구성을 위해 작업하기 위해 각 척도의 깊이 맵을 제한한다고 주장한다.  

### 기타 손실

또한 저자는 평균 정규화된 역 깊이 맵 값과 입력/대상 영상 사이에 에지 인식 평활성 손실을 사용한다. 이렇게 하면 모델이 날카로운 모서리를 학습하고 노이즈를 완화시킬 수 있습니다.

최종 손실 함수는 다음과 같습니다.
![The final loss function in [1] which is averaged over each pixel, scale and batch.](paper_7.png)  
> The final loss function in [1] which is averaged over each pixel, scale and batch.  

### 결과.

저자는 주행 시퀀스를 포함하는 세 가지 데이터 세트에서 모델을 비교했다. 그들의 방법은 모든 실험에서 거의 모든 다른 방법들보다 우수했다. 이러한 성능 예는 다음 이미지에 나와 있습니다.  
![Image from [1] GitHub repository: https://github.com/nianticlabs/monodepth2/](paper_8.gif)  
> Image from [1] GitHub repository: https://github.com/nianticlabs/monodepth2/  

**더 다양한 정보는 [여기](https://arxiv.org/pdf/1806.01260.pdf)를 클릭하세요**

# Monodepth2 Extension: Struct2Depth
## 객체 모션 모델링

Google Brain의 저자들은 Monodepth2를 더욱 확장한 [3]를 출판했다. 이들은 전체 이미지 대신 개별 객체의 예측 움직임에 의해 이전부터 포즈 네트워크를 개선한다. 따라서 재구성된 영상이 단일 투영인 대신, 이제 일련의 투영들이 결합됩니다. 이들은 두 가지 모델, 즉 객체 모션 모델과 ego-motion 네트워크를 통해 이를 수행한다(이전 섹션에서 설명한 포즈 네트워크와 유사). 단계는 다음과 같습니다.  
1. 잠재적으로 움직이는 물체의 분할을 포착하기 위해 사전 훈련된 마스크 R-CNN [2]가 적용된다.  
2. 2진수 마스크는 정적 영상(프레임 -1, 프레임 0 및 프레임 +1)에서 잠재적으로 움직이는 객체를 제거하는 데 사용됩니다.  
3. 마스크된 영상이 ego-motion 네트워크로 전송되고 프레임 -1과 0, 프레임 0과 +1 사이의 변환 매트릭스가 출력됩니다.  

![Sample output for Mask R-CNN [2]. Image from [2].](paper_9.png)
> Sample output for Mask R-CNN [2]. Image from [2].  

![The masking process to extract the static background followed by the ego-motion transformation matrix without objects that move. Equation from [3].](paper_10.png)
> The masking process to extract the static background followed by the ego-motion transformation matrix without objects that move. Equation from [3].  

1. 3단계의 결과 자기운동 변환 매트릭스를 사용하여 프레임-1과 프레임 +1에 적용하여 뒤틀린 프레임 0을 구합니다.  
2. 3단계의 결과로 생성된 ego-motion 변환 매트릭스를 사용하여 잠재적으로 움직이는 개체의 분할 마스크에 적용하여 프레임 1 및 프레임 +1에 대해 프레임 0에 대해 뒤틀린 분할 마스크를 얻습니다.  
3. 이진 마스크는 뒤틀린 분할 마스크와 연결된 픽셀을 유지하는 데 사용됩니다.  
4. 마스크된 영상이 뒤틀린 영상과 결합되어 예측된 객체 모션을 출력하는 객체 모션 모델로 전달됩니다.  
![The object motion model for one object. Equation from [3].](paper_11.png)
> The object motion model for one object. Equation from [3].  

그 결과는 사물의 외형 변화를 "설명"하기 위해 카메라가 어떻게 움직여야 하는지를 나타낸 것이다. 그런 다음 우리는 객체 모션 모델링 프로세스의 4단계로부터 결과 모션 모델에 따라 객체를 이동하기를 원한다. 마지막으로, 뒤틀린 물체 움직임을 뒤틀린 정적 배경과 결합하여 마지막 뒤틀림을 구합니다.

![Equation from [3].](paper_11.png)
> Equation from [3].  

![Image from [5]](paper_12.png)
> Image from [5]  

### 학습 개체 척도

Monoddepth2는 자동 마스크 기술을 통해 카메라와 동일한 속도로 움직이는 정적 물체 또는 물체의 문제를 해결하지만, 이러한 저자는 실제로 물체 운동의 모델링을 개선하기 위해 물체 척도를 인식하도록 모델을 훈련시킬 것을 제안한다.

![구조체 2 깊이에서 가져온 이미지입니다. 가운데 열은 카메라의 동일한 속도로 움직이는 물체에 무한 깊이가 할당되는 문제를 보여 줍니다. 세 번째 열에는 이를 개선하는 방법이 나와 있습니다.](paper_13.png)
> Struct2Depth에서 가져온 이미지입니다. 가운데 열은 카메라의 동일한 속도로 움직이는 물체에 무한 깊이가 할당되는 문제를 보여 줍니다. 세 번째 열에는 이를 개선하는 방법이 나와 있습니다.  

이들은 주택과 같은 객체의 범주에 따라 각 객체의 척도에 대한 손실을 정의합니다. 객체 척도의 지식을 기반으로 깊이를 제한하는 것을 목표로 한다. 손실은 카메라 초점 길이, 개체 범주를 기준으로 이전 높이, 이미지 내 분할된 개체의 실제 높이를 사용하여 계산된 이미지 내 개체의 출력 깊이 맵과 대략적인 깊이 맵 간의 차이입니다. 두 맵 모두 대상 이미지의 평균 깊이로 조정됩니다.  

![가운데 행은 [3]의 결과를 보여주는 반면, 세 번째 행에는 실측 결과가 표시됩니다. [5]의 이미지입니다.](paper_14.png)
> 가운데 행은 [3]의 결과를 보여주는 반면, 세 번째 행에는 실측 결과가 표시됩니다. [5]의 이미지입니다.  

### 결과.

[3]에서 설명한 확장자는 Monoddepth2 모델과 직접 비교되었으며 상당한 향상을 보였다.  

![가운데 행은 [3]의 결과를 보여주는 반면, 세 번째 행에는 실측 결과가 표시됩니다. [5]의 이미지입니다.](paper_15.png)
> 가운데 행은 [3]의 결과를 보여주는 반면, 세 번째 행에는 실측 결과가 표시됩니다. [5]의 이미지입니다.  

### 요약

자율 주행에서 깊이 추정의 일반적인 방법은 카메라 두 대 또는 LiDAR 깊이 센서가 필요한 스테레오 이미지 쌍을 사용하는 것이다. 그러나 이러한 것들은 비용이 많이 들고 항상 이용 가능한 것은 아니다. 여기서 설명하는 방법은 한 이미지의 깊이를 예측하고 일련의 이미지에 대해서만 학습하는 딥 러닝 모델을 훈련할 수 있다. 자율주행 연구에 좋은 성능과 큰 미래를 보여준다.

모델을 직접 사용해 보기 위해 두 문서 모두 아래에 저장소가 있습니다.  

Monodepth2: https://github.com/nianticlabs/monodepth2  
Struct2Depth: https://github.com/tensorflow/models/tree/master/research/struct2depth

### References

[1] Godard, C., Mac Aodha, O., Firman, M., & Brostow, G. (2018). Digging into self-supervised monocular depth estimation. arXiv preprint arXiv:1806.01260.

[2] Olaf Ronneberger, Philipp Fischer, and Thomas Brox. U-Net: Convolutional networks for biomedical image segmen-tation. InMICCAI, 2015.

[3] Vincent Casser, Soeren Pirk, Reza Mahjourian, Anelia Angelova: Depth Prediction Without the Sensors: Leveraging Structure for Unsupervised Learning from Monocular Videos. Thirty-Third AAAI Conference on Artificial Intelligence (AAAI’19).

[4] He, K., Gkioxari, G., Dollár, P., & Girshick, R. (2017). Mask r-cnn. In Proceedings of the IEEE international conference on computer vision (pp. 2961–2969).

[5] Vincent Casser, Soeren Pirk, Reza Mahjourian, Anelia Angelova: Unsupervised Monocular Depth and Ego-motion Learning with Structure and Semantics. CVPR Workshop on Visual Odometry & Computer Vision Applications Based on Location Clues (VOCVALC), 2019